In [1]:
import anndata as ad
import os
import senepy as sp
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random 

random.seed(42)

# Set working directory
os.chdir('/fs/scratch/PAS2598/Morales/CSF_workspace/csf')

# Check current working directory
print(os.getcwd())

/users/PAS2598/jmorales558/.conda/envs/senepy/lib/python3.9/site-packages/senepy/load_hubs.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/users/PAS2598/jmorales558/.local/lib/python3.9/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/users/PAS2598/jmorales558/.local/lib/python3.9/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/users/PAS2598/jmorales558/.local/lib/python3.9/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_

/fs/scratch/PAS2598/Morales/CSF_workspace/csf


In [2]:
# Load your h5ad file
cd8 = sc.read_h5ad('cd8_scored_7-29-25.h5ad')

# Check what you loaded
print(cd8)
print(f"Number of cells: {cd8.n_obs}")
print(f"Number of genes: {cd8.n_vars}")

# View the metadata
print(cd8.obs.head())

AnnData object with n_obs × n_vars = 8386 × 12215
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'percent.mito_log10', 'nCount_RNA_log10', 'nFeature_RNA_log10', 'nCount_RNA_log2', 'nFeature_RNA_log2', 'scaled_mito', 'scaled_nCount_RNA', 'sample', 'study', 'sex', 'organ', 'disease', 'dataset', 'disease_group', 'batch', 'cell_type', 'disease_group_comb', 'age', 'age_comparison', 'sen_score', 'putative_sen'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw_counts'
Number of cells: 8386
Number of genes: 12215
          nCount_RNA  nFeature_RNA  percent.mito  percent.mito_log10  \
319595-0      2704.0          1205      0.056583            0.023904   
319596-0      2858.0           890      0.053534            0.022649   
319597-0      2420.0          1145      0.058264            0.024594   
319598-0      1788.0           758 

In [3]:
# Check the merged object
print(cd8)
print(cd8.obs['cell_type'].value_counts())

AnnData object with n_obs × n_vars = 8386 × 12215
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'percent.mito_log10', 'nCount_RNA_log10', 'nFeature_RNA_log10', 'nCount_RNA_log2', 'nFeature_RNA_log2', 'scaled_mito', 'scaled_nCount_RNA', 'sample', 'study', 'sex', 'organ', 'disease', 'dataset', 'disease_group', 'batch', 'cell_type', 'disease_group_comb', 'age', 'age_comparison', 'sen_score', 'putative_sen'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw_counts'
cell_type
CD8 T cells    8386
Name: count, dtype: int64


In [4]:
print(cd8.obs['putative_sen'].value_counts())

putative_sen
0    8261
1     125
Name: count, dtype: int64


In [5]:
# Count cells per sample
print("Cells per sample:")
print(cd8.obs['sample'].value_counts().sort_index())

# Or get it as a sorted table
sample_counts = cd8.obs['sample'].value_counts().sort_index()
print(f"\nTotal samples: {len(sample_counts)}")
print(f"Cells per sample range: {sample_counts.min()} - {sample_counts.max()}")
print(f"Mean cells per sample: {sample_counts.mean():.1f}")

Cells per sample:
sample
GSM3984199     184
GSM3984201     170
GSM3984202     232
GSM3984205     270
GSM3984209     104
GSM3984211     152
GSM3984215     189
GSM3984216     138
GSM4104132    2120
GSM4104133      97
GSM4208773     272
GSM4208774     356
GSM4208775     412
GSM4208777     150
GSM4208778     214
GSM4208779     305
GSM4208780     217
GSM4404055     109
IIH45044      2215
IIH68490       444
IIH85037        36
Name: count, dtype: int64

Total samples: 21
Cells per sample range: 36 - 2215
Mean cells per sample: 399.3


In [6]:
grouping_vars = ['sample', 'putative_sen']

def pseudobulk_adata(adata, groupby_vars, layer='raw_counts'):
    """
    Pseudobulk AnnData object by specified grouping variables using raw_counts layer
    """
    # Check if raw_counts layer exists
    if layer not in adata.layers:
        raise ValueError(f"Layer '{layer}' not found in AnnData object. Available layers: {list(adata.layers.keys())}")
    
    print(f"Using layer: '{layer}' for pseudobulking")
    
    # Create grouping key
    adata.obs['pseudobulk_id'] = adata.obs[groupby_vars].astype(str).agg('_'.join, axis=1)
    
    # Get the raw count matrix specifically from raw_counts layer
    raw_counts_matrix = adata.layers['raw_counts']
    print(f"Raw counts matrix shape: {raw_counts_matrix.shape}")
    print(f"Raw counts matrix type: {type(raw_counts_matrix)}")
    
    # Convert to dense array if sparse
    if hasattr(raw_counts_matrix, 'toarray'):
        raw_counts_dense = raw_counts_matrix.toarray()
    else:
        raw_counts_dense = raw_counts_matrix
    
    # Create pseudobulk matrix using raw counts
    pseudobulk_df = pd.DataFrame(raw_counts_dense, 
                                index=adata.obs_names, 
                                columns=adata.var_names)
    pseudobulk_df['pseudobulk_id'] = adata.obs['pseudobulk_id'].values
    
    # Sum raw counts for each pseudobulk group
    print("Summing raw counts by pseudobulk groups...")
    pseudobulk_counts = pseudobulk_df.groupby('pseudobulk_id').sum()
    
    # Create metadata for pseudobulk samples
    # Keep all relevant sample-level metadata
    metadata_cols = groupby_vars + ['sex', 'disease', 'age', 'organ', 'dataset', 'disease_group', 'batch']
    available_cols = [col for col in metadata_cols if col in adata.obs.columns]
    
    pseudobulk_meta = adata.obs.groupby('pseudobulk_id')[available_cols].first()
    
    # Add cell count information
    pseudobulk_meta['n_cells'] = adata.obs.groupby('pseudobulk_id').size()
    
    # Add cell type composition for each pseudobulk sample
    print("Calculating cell type composition...")
    cell_type_counts = adata.obs.groupby('pseudobulk_id')['cell_type'].value_counts().unstack(fill_value=0)
    cell_type_props = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)
    
    # Add cell type proportions to metadata
    for cell_type in cell_type_props.columns:
        pseudobulk_meta[f'{cell_type}_prop'] = cell_type_props[cell_type]
        pseudobulk_meta[f'{cell_type}_count'] = cell_type_counts[cell_type]
    
    print(f"Pseudobulking complete!")
    print(f"Total raw counts in original data: {raw_counts_dense.sum():,.0f}")
    print(f"Total counts in pseudobulk data: {pseudobulk_counts.sum().sum():,.0f}")
    
    return pseudobulk_counts, pseudobulk_meta

# Verify raw_counts layer exists
print("Available layers in merged_adata:")
print(list(cd8.layers.keys()))

# Perform pseudobulking using raw_counts layer
print("\nPerforming pseudobulking by sample + senescence status using raw_counts layer...")
pb_counts, pb_meta = pseudobulk_adata(cd8, 
                                     groupby_vars=['sample', 'putative_sen'],
                                     layer='raw_counts')

print(f"\nCreated {pb_counts.shape[0]} pseudobulk samples with {pb_counts.shape[1]} genes")
print(f"Pseudobulk counts range: {pb_counts.min().min()} to {pb_counts.max().max()}")
print("\nPseudobulk sample breakdown:")
print(pb_meta[['putative_sen', 'n_cells']].groupby('putative_sen').agg(['count', 'mean']))

# Show sample of the data
print(f"\nFirst few pseudobulk samples:")
print(pb_meta[['putative_sen', 'n_cells']])

Available layers in merged_adata:
['raw_counts']

Performing pseudobulking by sample + senescence status using raw_counts layer...
Using layer: 'raw_counts' for pseudobulking
Raw counts matrix shape: (8386, 12215)
Raw counts matrix type: <class 'numpy.ndarray'>
Summing raw counts by pseudobulk groups...
Calculating cell type composition...
Pseudobulking complete!
Total raw counts in original data: 19,170,836
Total counts in pseudobulk data: 19,170,848

Created 41 pseudobulk samples with 12215 genes
Pseudobulk counts range: 0.0 to 327205.0

Pseudobulk sample breakdown:
             n_cells            
               count        mean
putative_sen                    
0                 21  393.380952
1                 20    6.250000

First few pseudobulk samples:
               putative_sen  n_cells
pseudobulk_id                       
GSM3984199_0              0      183
GSM3984199_1              1        1
GSM3984201_0              0      167
GSM3984201_1              1        3
GSM3984

In [7]:
# Show sample of the data
print(f"\nFirst few pseudobulk samples:")
print(pb_meta[['putative_sen', 'n_cells']])


First few pseudobulk samples:
               putative_sen  n_cells
pseudobulk_id                       
GSM3984199_0              0      183
GSM3984199_1              1        1
GSM3984201_0              0      167
GSM3984201_1              1        3
GSM3984202_0              0      228
GSM3984202_1              1        4
GSM3984205_0              0      267
GSM3984205_1              1        3
GSM3984209_0              0      103
GSM3984209_1              1        1
GSM3984211_0              0      151
GSM3984211_1              1        1
GSM3984215_0              0      179
GSM3984215_1              1       10
GSM3984216_0              0      137
GSM3984216_1              1        1
GSM4104132_0              0     2107
GSM4104132_1              1       13
GSM4104133_0              0       95
GSM4104133_1              1        2
GSM4208773_0              0      269
GSM4208773_1              1        3
GSM4208774_0              0      344
GSM4208774_1              1       12
GSM4208

In [9]:
def filter_pseudobulk_genes_only(counts, metadata, min_counts=3, min_sample_fraction=0.3):
    """
    Filter pseudobulk data for low count genes only
    - Keep genes with at least min_counts in at least min_sample_fraction of samples
    - Keep ALL samples (no sample filtering)
    """
    print(f"Before filtering: {counts.shape[0]} samples, {counts.shape[1]} genes")
    
    # Calculate minimum number of samples (30% of total samples)
    min_samples = int(np.ceil(counts.shape[0] * min_sample_fraction))
    print(f"Requiring ≥{min_counts} counts in ≥{min_samples} samples ({min_sample_fraction*100}% of {counts.shape[0]} samples)")
    
    # Filter genes ONLY - keep genes with ≥min_counts in ≥min_samples
    genes_pass_filter = (counts >= min_counts).sum(axis=0) >= min_samples
    
    counts_final = counts.loc[:, genes_pass_filter]
    meta_final = metadata  # Keep all sample metadata unchanged
    
    print(f"After gene filtering: {counts_final.shape[0]} samples, {counts_final.shape[1]} genes")
    print(f"Removed {counts.shape[1] - counts_final.shape[1]} genes with insufficient counts")
    print(f"Kept all {counts.shape[0]} samples")
    print(f"Senescent samples: {(meta_final['putative_sen'] == 1).sum()}")
    print(f"Non-senescent samples: {(meta_final['putative_sen'] == 0).sum()}")
    
    # Show filtering stats
    print(f"\nFiltering summary:")
    print(f"- Genes kept: {genes_pass_filter.sum():,} / {len(genes_pass_filter):,} ({genes_pass_filter.mean()*100:.1f}%)")
    print(f"- Samples kept: {counts_final.shape[0]} / {counts.shape[0]} (100%)")
    
    return counts_final, meta_final

# Apply gene-only filtering
pb_counts_filt, pb_meta_filt = filter_pseudobulk_genes_only(pb_counts, pb_meta, 
                                                           min_counts=3, 
                                                           min_sample_fraction=0.3)

# Show some examples of what got filtered
print(f"\nExample of count distribution for a few genes:")
sample_genes = pb_counts.columns[:5]  # first 5 genes
for gene in sample_genes:
    n_samples_with_counts = (pb_counts[gene] >= 3).sum()
    pct_samples = n_samples_with_counts / pb_counts.shape[0] * 100
    kept = gene in pb_counts_filt.columns
    print(f"{gene}: ≥3 counts in {n_samples_with_counts}/{pb_counts.shape[0]} samples ({pct_samples:.1f}%) - {'KEPT' if kept else 'FILTERED'}")

# Export raw filtered counts (genes as rows, samples as columns for DESeq2)
raw_counts_df = pd.DataFrame(pb_counts_filt.values.T,  # Transpose: genes as rows
                           index=pb_counts_filt.columns,   # gene names as row names
                           columns=pb_counts_filt.index)   # sample names as column names

raw_counts_df.to_csv('cd8_pseudobulk_raw_counts_for_deseq2.csv')

# Export metadata (samples as rows)
pb_meta_filt.to_csv('cd8_pseudobulk_metadata_for_deseq2.csv')

print(f"\n✅ Exported for DESeq2:")
print(f"   - Raw counts: cd8_pseudobulk_raw_counts_for_deseq2.csv ({raw_counts_df.shape[0]} genes × {raw_counts_df.shape[1]} samples)")
print(f"   - Metadata: cd8_pseudobulk_metadata_for_deseq2.csv ({pb_meta_filt.shape[0]} samples)")

Before filtering: 41 samples, 12215 genes
Requiring ≥3 counts in ≥13 samples (30.0% of 41 samples)
After gene filtering: 41 samples, 7126 genes
Removed 5089 genes with insufficient counts
Kept all 41 samples
Senescent samples: 20
Non-senescent samples: 21

Filtering summary:
- Genes kept: 7,126 / 12,215 (58.3%)
- Samples kept: 41 / 41 (100%)

Example of count distribution for a few genes:
A1BG: ≥3 counts in 23/41 samples (56.1%) - KEPT
A1BG-AS1: ≥3 counts in 13/41 samples (31.7%) - KEPT
A2M: ≥3 counts in 4/41 samples (9.8%) - FILTERED
A2M-AS1: ≥3 counts in 11/41 samples (26.8%) - FILTERED
AAAS: ≥3 counts in 19/41 samples (46.3%) - KEPT

✅ Exported for DESeq2:
   - Raw counts: pseudobulk_raw_counts_for_deseq2.csv (7126 genes × 41 samples)
   - Metadata: pseudobulk_metadata_for_deseq2.csv (41 samples)
